In [ ]:
# !git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
# %cd /content/Generative_Models/
# !git checkout lmdb

In [1]:
!pip -q install keras
!pip -q install colorlog
!pip -q install keras-radam
!pip -q install lmdb
!pip -q install Pillow

In [2]:
import os
os.chdir('../../Generative_Models/')
print(os.getcwd())

/home/azeghost/git/Generative_Models


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
images_dir = './data/.pokemon/'
validation_percentage = 30
valid_format = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [6]:
!ls -la ./data/.pokemon

total 28
drwxrwxr-x 3 azeghost azeghost  4096 Jun 26 22:40 .
drwxr-xr-x 6 azeghost azeghost  4096 Jun 26 22:40 ..
drwxrwxr-x 2 azeghost azeghost 20480 Jun 26 22:40 DS06


In [7]:
image_lists = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 1
)

  DEBUG    | Looking for images in 'DS06'
  INFO     | 809 file found


In [8]:
import numpy as np
import lmdb
import pickle
import math
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img

In [9]:
from utils.data_and_files.file_utils import get_file_path

In [10]:
class Pokemon_Image:
    def __init__(self, image, label):
        # Dimensions of image for reconstruction - not really necessary 
        # for this dataset, but some datasets may include images of 
        # varying sizes
        self.channels = image.shape[2]
        self.size = image.shape[:2]

        self.image = image.tobytes()
        self.label = label
    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32)
        return images.reshape(*self.size, self.channels)

In [11]:
import sys
def store_single_lmdb(filename, img , index, label, num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = Pokemon_Image(img, label)
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

Test 1

In [12]:

def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    print('Storing '+ str(num_images)+lmdb_dir+os.sep+'_{}'.format(category))
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]     
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        
        store_single_lmdb(index = index, filename=name, img = img,label = y ,num_images = num_images)

In [13]:
!rm -rf ./LMDB/
!mkdir ./LMDB/

In [14]:
# save_to_dir=None,
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB',category='training',target_size=None,color_mode='rgb',save_prefix='',save_format='png')
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB',category='validation',target_size=None,color_mode='rgb',save_prefix='',save_format='png')

Storing 591./LMDB/_training
Storing 218./LMDB/_validation


In [ ]:
# In  case we want each label to be a key then

def store_many_lmdb_labelaskey(filename, images, labels):
    """ Stores an array of images to LMDB.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    #str(lmdb_dir / f"{num_images}_lmdb")
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            value = Pokemon_Image(images[i], labels[i])
            key = labels[i]
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()


In [27]:
def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], []
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            pokemon_image = pickle.loads(data)
            images.append(pokemon_image.get_image())
            labels.append(pokemon_image.label)
    env.close()
    return images, labels

Test 2 Reading data


In [28]:
x, y = read_many_lmdb('./LMDB/_validation', 218)

In [29]:
print(len(x), len(y))

218 218


In [ ]:
import math
def read_many_lmdb_old(filename, num_images):
    """ Reads image from LMDB.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []
    #str(lmdb_dir / f"{num_images}_lmdb")
    for folder_indx in  range(math.ceil(num_images/200)):
      env = lmdb.open(filename+os.sep+'_{}'.format(float(folder_indx+1)), readonly=True)
      print(folder_indx)
      # Start a new read transaction
      with env.begin() as txn:
          # Read all images in one single transaction, with one lock
          # We could split this up into multiple transactions if needed
          if num_images- folder_indx*200 >200:
            max_index = 200 
          else:
            max_index = num_images - folder_indx*200
          
          for image_id in range(0,max_index):
              data = txn.get(f"{image_id:08}".encode("ascii"))
              # Remember that it's a CIFAR_Image object 
              # that is stored as the value
              pokemon_image = pickle.loads(data)
              # Retrieve the relevant bits
              images.append(pokemon_image.get_images())
              labels.append(pokemon_image.label)
      env.close()
    return images, labels

In [30]:
# Single Image Experiment

In [31]:
def store_many_lmdb_x(lmdb_dir, images, labels):
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    env = lmdb.open(lmdb_dir, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            
            value = images[i].tobytes()
            
            key = f"{i:08}"
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

In [32]:

def read_many_lmdb_x(lmdb_dir, num_images):

    images, labels = [], []
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            image = pickle.loads(data)
            image = np.frombuffer(image, dtype=np.float32)
            image = image.reshape((400,400,3))
            # Retrieve the relevant bits
            images.append(image)
#             labels.append(cifar_image.label)
    env.close()
    return images, labels

In [33]:

def get_images(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    print('Storing '+ str(num_images)+lmdb_dir+os.sep+'_{}'.format(category))
    for index, _ in enumerate(image_lists[label_name][category]):
        if index < 3:
            img_path = get_file_path(image_lists,
                                      label_name,
                                      index,
                                      image_dir,
                                      category)
            img = img_to_array(
                            load_img(
                            img_path,
                            grayscale=color_mode=='grayscale',
                            target_size=target_size
                            )
                        )
            name, _ = os.path.splitext(img_path)
            vid_img_arr = name.split(sep=os.sep)[-1:]

            if (y is None):
                  y = [np.array(vid_img_arr)]
            else:
                  y = np.append(y, [np.array(vid_img_arr)], 0)
            if (X is None):
                  X = [np.array(img)]
            else:
                  X = np.append(X, [img], 0)
    return X ,y      

In [34]:
x, y = get_images(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='validation',target_size=None,color_mode='rgb'
           ,save_prefix='',save_format='png')
print(x.shape, y.shape)

Storing 218./LMDB/_validation
(3, 400, 400, 3) (3, 1)


In [35]:
x.dtype

dtype('float32')

In [36]:

store_many_lmdb_x('test', x , y)

In [37]:
xt,yt = read_many_lmdb_x('test', 3)

In [38]:
print(x == xt)

[[[[ True  True  True]
   [ True  True  True]
   [ True  True  True]
   ...
   [ True  True  True]
   [ True  True  True]
   [ True  True  True]]

  [[ True  True  True]
   [ True  True  True]
   [ True  True  True]
   ...
   [ True  True  True]
   [ True  True  True]
   [ True  True  True]]

  [[ True  True  True]
   [ True  True  True]
   [ True  True  True]
   ...
   [ True  True  True]
   [ True  True  True]
   [ True  True  True]]

  ...

  [[ True  True  True]
   [ True  True  True]
   [ True  True  True]
   ...
   [ True  True  True]
   [ True  True  True]
   [ True  True  True]]

  [[ True  True  True]
   [ True  True  True]
   [ True  True  True]
   ...
   [ True  True  True]
   [ True  True  True]
   [ True  True  True]]

  [[ True  True  True]
   [ True  True  True]
   [ True  True  True]
   ...
   [ True  True  True]
   [ True  True  True]
   [ True  True  True]]]


 [[[ True  True  True]
   [ True  True  True]
   [ True  True  True]
   ...
   [ True  True  True]
   [ True 